# Reddit API Web Scraper
I started building a true web scraper using BeautifulSoup, and later discovered Reddit has an API to grab content, so I'm going to take advantage of that instead.

In [29]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import requests
import praw

C:\Program Files\Anaconda\lib\site-packages\pandas\__init__.py:7: DeprecationWarning: bad escape \s
  from pandas import hashtable, tslib, lib


Before I use praw to grab the comments, I'll scrap the front page with beautiful soup so I can grab the IDs for each of the threads

In [131]:
# This is the webpage I want to scrape
url = requests.get('http://www.reddit.com/r/summonerschool/',headers={"User-Agent": "Comment Scraper by zachturn"})

# Load in the HTML data
data = url.text
soup = BeautifulSoup(data,"lxml")


threads = soup.select("div#siteTable div.thing")

ids = []
for thread in threads:
    ids.append(re.search('_(.*)',thread['data-fullname']).group(1))

C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:240: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


Don't need the code below (for now)

# PRAW

In [253]:
start = time.clock()
r = praw.Reddit('Comment scraping attempt using praw by /u/zachturn')
threads = {}
for _id in ids:
    submission = r.get_submission(submission_id=_id,comment_limit=None)
    threads.setdefault(submission.title,{'sub_object':submission,'comments':[]})

for k,v in threads.items():
    v['sub_object'].replace_more_comments(limit=None,threshold=0)
    flat_comments = praw.helpers.flatten_tree(v['sub_object'].comments)
    for comment in flat_comments:
        v['comments'].append(comment.body)
runtime = time.clock() - start